# ASYNC AWAIT

Асинхронное программирование позволяет программам эффективно обрабатывать множество задач одновременно, не блокируя основной поток выполнения во время ожидания операций. Этот метод улучшает производительность, позволяя использовать время ожидания для выполнения других задач и оптимизируя использование ресурсов. 

Важно уметь различать задачи, связанные с вводом-выводом (I/O-bound tasks), и задачи, требующие интенсивных вычислений (CPU-bound tasks).

I/O-bound операции включают в себя время ожидания для завершения таких действий, как выполнение сетевых запросов или чтение данных с диска. В этих случаях применение асинхронного программирования оказывается особенно эффективным. 

CPU-bound операции представляют собой вычисления, требующие значительных ресурсов процессора. Хотя асинхронность не является единственным способом ускорения вычислительных операций, такие задачи могут быть оптимизированы в рамках асинхронной программы. Например, можно эффективно задеплоить ML-модель в асинхронный сервис.

Python поддерживает асинхронное программирование с помощью конструкций async и await. Библиотека asyncio предоставляет необходимый фреймворк для написания асинхронного кода. Она использует такие концепции, как корутины (функции, определенные с помощью async def), задачи (запланированные корутины) и цикл событий (основа механизма выполнения asyncio), для управления асинхронными операциями.

# Задание
Вам дан код синхронного FastAPI сервиса (API), предназначенного для парсинга HTML-страниц. Сервис принимает URL (функция /parse_url/), по которому он выполняет HTTP-запрос, а затем имитирует процесс парсинга страницы. Время, затрачиваемое на парсинг, варьируется: обычно это 0.1 секунды, но с вероятностью 10% это время увеличивается в несколько раз, что имитирует случаи более сложного и длительного парсинга.

В функции run_test происходит симуляция нагрузки на API при обработке множества запросов одновременно. Поскольку сервис является синхронным, он будет обрабатывать каждый входящий запрос последовательно, то есть начнет обрабатывать следующий запрос только после того, как закончит обработку предыдущего. Это может привести к простою при обработке большого количества запросов, особенно из-за периодических "зависаний" в парсинге.

Симуляция в данном случае означает тестирование сервиса в тестовой среде (TestClient) без необходимости запуска сервера на компьютере. Такой метод подходит для написания юнит-тестов и других видов тестирования API.

In [2]:
import random
import time

import httpx
from fastapi import FastAPI
from fastapi.testclient import TestClient

app = FastAPI()


@app.post("/parse_url/")
def parse_url(url: str) -> str:
    try:
        with httpx.Client() as client:
            r = client.get(url)
            r.raise_for_status()

            parse_time = 0.1 * random.randint(5, 10) if random.random() < 0.1 else 0.1
            time.sleep(parse_time)

            return f"Parsed {url}"
    except Exception as e:
        return f"Error fetching {url}: {e}"


def run_test(n_requests: int) -> float:
    url = "https://httpbin.org/"

    with TestClient(app) as client:
        ts = time.time()
        for _ in range(n_requests):
            _ = client.post("/parse_url/", params={"url": url})
        return time.time() - ts


if __name__ == "__main__":
    t = run_test(n_requests=100)
    print(f"Time taken: {t} seconds")


Time taken: 73.88819169998169 seconds


Перепишите сервис:

Перепишите сервис, сделав его асинхронным. Функция parse_url асинхронно извлекает содержимое по-указанному URL, включая случайную задержку, чтобы имитировать время обработки, а run_test симулирует нагрузку, выполняя множество асинхронных запросов к API одновременно и возвращая общее время выполнения.

In [9]:
import asyncio
import random
import time

import httpx
from fastapi import FastAPI

app = FastAPI()


@app.post("/parse_url/")
# async parse_url
async def parse_url(url: str) -> str:
    """Asynchronous function to parse url"""
    try:
        async with httpx.AsyncClient() as client:
            r = await client.get(url)
            r.raise_for_status()

            parse_time = 0.1 * random.randint(5, 10) if random.random() < 0.1 else 0.1
            await asyncio.sleep(parse_time)

            return f"Parsed {url}"
    except Exception as e:
        return f"Error fetching {url}: {e}"


# async run_test
async def run_test(n_requests: int) -> float:
    """ Asynchronous function to run test"""
    url = "https://httpbin.org/"

    async with httpx.AsyncClient() as client:
        ts = time.time()
        tasks = [client.post("http://localhost:8000/parse_url/", json={"url": url}) for _ in range(n_requests)]
        await asyncio.gather(*tasks)
        return time.time() - ts

if __name__ == "__main__":
    t = asyncio.run(run_test(n_requests=100))
    print(f"Time taken: {t} seconds")


<frozen genericpath>:77: RuntimeWarning: coroutine 'run_test' was never awaited


RuntimeError: asyncio.run() cannot be called from a running event loop

In [10]:
import asyncio
import random
import time

import httpx
from fastapi import FastAPI

app = FastAPI()


@app.post("/parse_url/")
async def parse_url(url: str):
    try:
        async with httpx.AsyncClient() as client:
            r = await client.get(url)
            r.raise_for_status()

            parse_time = 0.5 if random.random() < 0.1 else 0.1
            await asyncio.sleep(parse_time)

            return f"Parsed {url}"
    except Exception as e:
        return f"Error fetching {url}: {e}"


async def run_test(n_requests: int):
    url = "https://httpbin.org/"

    async with httpx.AsyncClient(
        app=app,
        base_url="http://test",
    ) as client:
        tasks = []
        for _ in range(n_requests):
            tasks.append(client.post("/parse_url/", params={"url": url}))

        ts = time.time()
        _ = await asyncio.gather(*tasks)
        return time.time() - ts


if __name__ == "__main__":
    t = asyncio.run(run_test(n_requests=100))
    print(f"Time taken: {t} seconds")


RuntimeError: asyncio.run() cannot be called from a running event loop